# Import Libraries, Datesets, Models

In [7]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import eli5
from PIL import Image

from src.constants import LABELS
from src.preprocessing_functions import create_generator

from keras.models import load_model

from eli5.keras import explain_prediction_keras
from eli5 import show_prediction
from IPython.display import display

In [2]:
# Load paths in a dict for train, test, valid folders with wildfire and nowildfire images 

images_path = '/Users/silvanoquarto/Desktop/PROJECTS/Data_Wildfire_Project'
splits = ['train', 'test', 'valid']
labels = ['wildfire', 'nowildfire']

image_paths = {f"{split}_{label}": [] for split in splits for label in labels}

for split in splits:
    for label in labels:
        path = os.path.join(images_path, split, label)
        if os.path.exists(path):  
            image_files = os.listdir(path)
            full_paths = [os.path.join(path, img) for img in image_files]
            image_paths[f"{split}_{label}"].extend(full_paths)
            print(f"- {split.capitalize()} set for {label} uploaded correctly!!")
        else:
            print(f"Warning: {path} does not exist :(")

- Train set for wildfire uploaded correctly!!
- Train set for nowildfire uploaded correctly!!
- Test set for wildfire uploaded correctly!!
- Test set for nowildfire uploaded correctly!!
- Valid set for wildfire uploaded correctly!!
- Valid set for nowildfire uploaded correctly!!


In [3]:
# Data augumentation and normalization
 
print('Data augumentation and normalization for training...')
train_generator = create_generator(images_path, 'train', num_output=LABELS)

print('Data augumentation and normalization for validation...')
validation_generator = create_generator(images_path, 'valid', num_output=LABELS)

print('Data augumentation and normalization for test...')
test_generator = create_generator(images_path, 'test', num_output=LABELS)

Data augumentation and normalization for training...
Found 30249 images belonging to 2 classes.
Data augumentation and normalization for validation...
Found 6300 images belonging to 2 classes.
Data augumentation and normalization for test...
Found 6299 images belonging to 2 classes.


In [9]:
# Load Model

model = load_model('Saved_Models/best_model.keras')

# ELI5 Explainability using Tensorflow

In [67]:
# Get the image you want 

batch_images, batch_labels = next(test_generator)
sample_image = batch_images[0]  

sample_image_pil = Image.fromarray((sample_image * 255).astype(np.uint8))

doc = np.array(sample_image_pil)  

doc = np.expand_dims(doc, axis=0)  


In [68]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 350, 350, 8)    │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 175, 175, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 175, 175, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 87, 87, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 87, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 59168)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    15,147,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,461,432 (173.42 MB)

 Trainable params: 15,153,810 (57.81 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 30,307,622 (115.61 MB)

In [ ]:
# Identify a suitable convolutional layer for Grad-CAM

layer_name = 'conv2d_2'  

# Apply Grad-CAM using ELI5

explanation = eli5.keras.explain_prediction_keras(model, doc, layer= layer_name)

In [ ]:
# Extract Grad-CAM heatmap from the explanation

heatmap = explanation.targets[0].heatmap  

plt.imshow(sample_image) 
plt.imshow(heatmap, alpha=0.6, cmap='jet') 
plt.axis('off')
plt.show()